# 9. Implement PKCS#7 padding

A block cipher transforms a fixed-sized block (usually 8 or 16 bytes) of plaintext into ciphertext. But we almost never want to transform a single block; we encrypt irregularly-sized messages.

One way we account for irregularly-sized messages is by padding, creating a plaintext that is an even multiple of the blocksize. The most popular padding scheme is called PKCS#7.

So: pad any block to a specific block length, by appending the number of bytes of padding to the end of the block. For instance,

"YELLOW SUBMARINE"
... padded to 20 bytes would be:

"YELLOW SUBMARINE\x04\x04\x04\x04"

In [1]:
test = b"YELLOW SUBMARINE"
testResult = b"YELLOW SUBMARINE\x04\x04\x04\x04"

def pkcs7Padding(data, blockSize):
    overhang = len(data) % blockSize
    paddingLength = 0 if overhang == 0 else blockSize - overhang
    # print(f"len(data): {len(data)}, blocksize: {blockSize} paddinglength: {paddingLength}")
    padding = bytes([4] * paddingLength) #padding with 00000100
    output = data + padding
    return output if len(output) >= blockSize else bytes([4] * blockSize)


paddedData = pkcs7Padding(test, 20)
assert paddedData == testResult
paddedData

b'YELLOW SUBMARINE\x04\x04\x04\x04'

# 10. Implement CBC mode
CBC mode is a block cipher mode that allows us to encrypt irregularly-sized messages, despite the fact that a block cipher natively only transforms individual blocks.

In CBC mode, each ciphertext block is added to the next plaintext block before the next call to the cipher core.

The first plaintext block, which has no associated previous ciphertext block, is added to a "fake 0th ciphertext block" called the initialization vector, or IV.

Implement CBC mode by hand by taking the ECB function you wrote earlier, making it encrypt instead of decrypt (verify this by decrypting whatever you encrypt to test), and using your XOR function from the previous exercise to combine them.

The data is intelligible (somewhat) when CBC decrypted against "YELLOW SUBMARINE" with an IV of all ASCII 0 (\x00\x00\x00 &c)

In [2]:
#@title data
ex10data = """CRIwqt4+szDbqkNY+I0qbNXPg1XLaCM5etQ5Bt9DRFV/xIN2k8Go7jtArLIy
P605b071DL8C+FPYSHOXPkMMMFPAKm+Nsu0nCBMQVt9mlluHbVE/yl6VaBCj
NuOGvHZ9WYvt51uR/lklZZ0ObqD5UaC1rupZwCEK4pIWf6JQ4pTyPjyiPtKX
g54FNQvbVIHeotUG2kHEvHGS/w2Tt4E42xEwVfi29J3yp0O/TcL7aoRZIcJj
MV4qxY/uvZLGsjo1/IyhtQp3vY0nSzJjGgaLYXpvRn8TaAcEtH3cqZenBoox
BH3MxNjD/TVf3NastEWGnqeGp+0D9bQx/3L0+xTf+k2VjBDrV9HPXNELRgPN
0MlNo79p2gEwWjfTbx2KbF6htgsbGgCMZ6/iCshy3R8/abxkl8eK/VfCGfA6
bQQkqs91bgsT0RgxXSWzjjvh4eXTSl8xYoMDCGa2opN/b6Q2MdfvW7rEvp5m
wJOfQFDtkv4M5cFEO3sjmU9MReRnCpvalG3ark0XC589rm+42jC4/oFWUdwv
kzGkSeoabAJdEJCifhvtGosYgvQDARUoNTQAO1+CbnwdKnA/WbQ59S9MU61Q
KcYSuk+jK5nAMDot2dPmvxZIeqbB6ax1IH0cdVx7qB/Z2FlJ/U927xGmC/RU
FwoXQDRqL05L22wEiF85HKx2XRVB0F7keglwX/kl4gga5rk3YrZ7VbInPpxU
zgEaE4+BDoEqbv/rYMuaeOuBIkVchmzXwlpPORwbN0/RUL89xwOJKCQQZM8B
1YsYOqeL3HGxKfpFo7kmArXSRKRHToXuBgDq07KS/jxaS1a1Paz/tvYHjLxw
Y0Ot3kS+cnBeq/FGSNL/fFV3J2a8eVvydsKat3XZS3WKcNNjY2ZEY1rHgcGL
5bhVHs67bxb/IGQleyY+EwLuv5eUwS3wljJkGcWeFhlqxNXQ6NDTzRNlBS0W
4CkNiDBMegCcOlPKC2ZLGw2ejgr2utoNfmRtehr+3LAhLMVjLyPSRQ/zDhHj
Xu+Kmt4elmTmqLgAUskiOiLYpr0zI7Pb4xsEkcxRFX9rKy5WV7NhJ1lR7BKy
alO94jWIL4kJmh4GoUEhO+vDCNtW49PEgQkundV8vmzxKarUHZ0xr4feL1ZJ
THinyUs/KUAJAZSAQ1Zx/S4dNj1HuchZzDDm/nE/Y3DeDhhNUwpggmesLDxF
tqJJ/BRn8cgwM6/SMFDWUnhkX/t8qJrHphcxBjAmIdIWxDi2d78LA6xhEPUw
NdPPhUrJcu5hvhDVXcceZLa+rJEmn4aftHm6/Q06WH7dq4RaaJePP6WHvQDp
zZJOIMSEisApfh3QvHqdbiybZdyErz+yXjPXlKWG90kOz6fx+GbvGcHqibb/
HUfcDosYA7lY4xY17llY5sibvWM91ohFN5jyDlHtngi7nWQgFcDNfSh77TDT
zltUp9NnSJSgNOOwoSSNWadm6+AgbXfQNX6oJFaU4LQiAsRNa7vX/9jRfi65
5uvujM4ob199CZVxEls10UI9pIemAQQ8z/3rgQ3eyL+fViyztUPg/2IvxOHv
eexE4owH4Fo/bRlhZK0mYIamVxsRADBuBlGqx1b0OuF4AoZZgUM4d8v3iyUu
feh0QQqOkvJK/svkYHn3mf4JlUb2MTgtRQNYdZKDRgF3Q0IJaZuMyPWFsSNT
YauWjMVqnj0AEDHh6QUMF8bXLM0jGwANP+r4yPdKJNsoZMpuVoUBJYWnDTV+
8Ive6ZgBi4EEbPbMLXuqDMpDi4XcLE0UUPJ8VnmO5fAHMQkA64esY2QqldZ+
5gEhjigueZjEf0917/X53ZYWJIRiICnmYPoM0GSYJRE0k3ycdlzZzljIGk+P
Q7WgeJhthisEBDbgTuppqKNXLbNZZG/VaTdbpW1ylBv0eqamFOmyrTyh1APS
Gn37comTI3fmN6/wmVnmV4/FblvVwLuDvGgSCGPOF8i6FVfKvdESs+yr+1AE
DJXfp6h0eNEUsM3gXaJCknGhnt3awtg1fSUiwpYfDKZxwpPOYUuer8Wi+VCD
sWsUpkMxhhRqOBKaQaBDQG+kVJu6aPFlnSPQQTi1hxLwi0l0Rr38xkr+lHU7
ix8LeJVgNsQdtxbovE3i7z3ZcTFY7uJkI9j9E0muDN9x8y/YN25rm6zULYaO
jUoP/7FQZsSgxPIUvUiXkEq+FU2h0FqAC7H18cr3Za5x5dpw5nwawMArKoqG
9qlhqc34lXV0ZYwULu58EImFIS8+kITFuu7jOeSXbBgbhx8zGPqavRXeiu0t
bJd0gWs+YgMLzXtQIbQuVZENMxJSZB4aw5lPA4vr1fFBsiU4unjOEo/XAgwr
Tc0w0UndJFPvXRr3Ir5rFoIEOdRo+6os5DSlk82SBnUjwbje7BWsxWMkVhYO
6bOGUm4VxcKWXu2jU66TxQVIHy7WHktMjioVlWJdZC5Hq0g1LHg1nWSmjPY2
c/odZqN+dBBC51dCt4oi5UKmKtU5gjZsRSTcTlfhGUd6DY4Tp3CZhHjQRH4l
Zhg0bF/ooPTxIjLKK4r0+yR0lyRjqIYEY27HJMhZDXFDxBQQ1UkUIhAvXacD
WB2pb3YyeSQjt8j/WSbQY6TzdLq8SreZiuMWcXmQk4EH3xu8bPsHlcvRI+B3
gxKeLnwrVJqVLkf3m2cSGnWQhSLGbnAtgQPA6z7u3gGbBmRtP0KnAHWSK7q6
onMoYTH+b5iFjCiVRqzUBVzRRKjAL4rcL2nYeV6Ec3PlnboRzJwZIjD6i7WC
dcxERr4WVOjOBX4fhhKUiVvlmlcu8CkIiSnZENHZCpI41ypoVqVarHpqh2aP
/PS624yfxx2N3C2ci7VIuH3DcSYcaTXEKhz/PRLJXkRgVlWxn7QuaJJzDvpB
oFndoRu1+XCsup/AtkLidsSXMFTo/2Ka739+BgYDuRt1mE9EyuYyCMoxO/27
sn1QWMMd1jtcv8Ze42MaM4y/PhAMp2RfCoVZALUS2K7XrOLl3s9LDFOdSrfD
8GeMciBbfLGoXDvv5Oqq0S/OvjdID94UMcadpnSNsist/kcJJV0wtRGfALG2
+UKYzEj/2TOiN75UlRvA5XgwfqajOvmIIXybbdhxpjnSB04X3iY82TNSYTmL
LAzZlX2vmV9IKRRimZ2SpzNpvLKeB8lDhIyGzGXdiynQjFMNcVjZlmWHsH7e
ItAKWmCwNkeuAfFwir4TTGrgG1pMje7XA7kMT821cYbLSiPAwtlC0wm77F0T
a7jdMrLjMO29+1958CEzWPdzdfqKzlfBzsba0+dS6mcW/YTHaB4bDyXechZB
k/35fUg+4geMj6PBTqLNNWXBX93dFC7fNyda+Lt9cVJnlhIi/61fr0KzxOeX
NKgePKOC3Rz+fWw7Bm58FlYTgRgN63yFWSKl4sMfzihaQq0R8NMQIOjzuMl3
Ie5ozSa+y9g4z52RRc69l4n4qzf0aErV/BEe7FrzRyWh4PkDj5wy5ECaRbfO
7rbs1EHlshFvXfGlLdEfP2kKpT9U32NKZ4h+Gr9ymqZ6isb1KfNov1rw0KSq
YNP+EyWCyLRJ3EcOYdvVwVb+vIiyzxnRdugB3vNzaNljHG5ypEJQaTLphIQn
lP02xcBpMNJN69bijVtnASN/TLV5ocYvtnWPTBKu3OyOkcflMaHCEUgHPW0f
mGfld4i9Tu35zrKvTDzfxkJX7+KJ72d/V+ksNKWvwn/wvMOZsa2EEOfdCidm
oql027IS5XvSHynQtvFmw0HTk9UXt8HdVNTqcdy/jUFmXpXNP2Wvn8PrU2Dh
kkIzWhQ5Rxd/vnM2QQr9Cxa2J9GXEV3kGDiZV90+PCDSVGY4VgF8y7GedI1h
"""

In [3]:
#@title installing cryptography
!pip3 install cryptography --quiet

In [4]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

In [5]:
testString = b"hbdft3fsbnhlngbe"
testKey = b"yellow submarine"

def encryptAES_ECB(data, key):
    paddedData = pkcs7Padding(data, len(key))

    AESCipher = Cipher(algorithms.AES(key), modes.ECB())
    encryptor = AESCipher.encryptor()

    ciphertextBytes = encryptor.update(paddedData) + encryptor.finalize()
    return ciphertextBytes

def decryptAES_ECB(data, key):
    AESCipher = Cipher(algorithms.AES(key), modes.ECB())
    decryptor = AESCipher.decryptor()
    plaintextBytes = decryptor.update(data) + decryptor.finalize()
    return plaintextBytes

ct = encryptAES_ECB(testString, testKey)
recoveredPlaintext = decryptAES_ECB(ct, testKey)

assert recoveredPlaintext == testString
print(f"in: {testString}\nct: {ct}\npt: {recoveredPlaintext}")

in: b'hbdft3fsbnhlngbe'
ct: b'\xfag\xf9 8\x1d\xfa\xa7\xdbE\xfa\xdf}\xfb\x81e'
pt: b'hbdft3fsbnhlngbe'


In [6]:
#@title helper functions

def xorBytes(in1, in2):
  xor = bytes([x ^ y for (x,y) in zip(in1, in2)])
  return xor

def reshape(data, keySize):

  dataSize = len(data)

  out = []
  data = pkcs7Padding(data, keySize)

  startIndex = 0
  endIndex = keySize

  for i in range(0, dataSize, keySize):
    out += [data[startIndex : endIndex]]
    startIndex += keySize
    endIndex += keySize
  return out

def flatten(byteArray):
    flatData = b''
    for byte in byteArray:
        flatData += byte
    return flatData

def binary(data_bytes):
    return [bin(byte)[2:] for byte in data_bytes]

def integers(data_bytes):
    return [int(byte) for byte in data_bytes]


In [7]:
#@title data
ex10data = """CRIwqt4+szDbqkNY+I0qbNXPg1XLaCM5etQ5Bt9DRFV/xIN2k8Go7jtArLIy
P605b071DL8C+FPYSHOXPkMMMFPAKm+Nsu0nCBMQVt9mlluHbVE/yl6VaBCj
NuOGvHZ9WYvt51uR/lklZZ0ObqD5UaC1rupZwCEK4pIWf6JQ4pTyPjyiPtKX
g54FNQvbVIHeotUG2kHEvHGS/w2Tt4E42xEwVfi29J3yp0O/TcL7aoRZIcJj
MV4qxY/uvZLGsjo1/IyhtQp3vY0nSzJjGgaLYXpvRn8TaAcEtH3cqZenBoox
BH3MxNjD/TVf3NastEWGnqeGp+0D9bQx/3L0+xTf+k2VjBDrV9HPXNELRgPN
0MlNo79p2gEwWjfTbx2KbF6htgsbGgCMZ6/iCshy3R8/abxkl8eK/VfCGfA6
bQQkqs91bgsT0RgxXSWzjjvh4eXTSl8xYoMDCGa2opN/b6Q2MdfvW7rEvp5m
wJOfQFDtkv4M5cFEO3sjmU9MReRnCpvalG3ark0XC589rm+42jC4/oFWUdwv
kzGkSeoabAJdEJCifhvtGosYgvQDARUoNTQAO1+CbnwdKnA/WbQ59S9MU61Q
KcYSuk+jK5nAMDot2dPmvxZIeqbB6ax1IH0cdVx7qB/Z2FlJ/U927xGmC/RU
FwoXQDRqL05L22wEiF85HKx2XRVB0F7keglwX/kl4gga5rk3YrZ7VbInPpxU
zgEaE4+BDoEqbv/rYMuaeOuBIkVchmzXwlpPORwbN0/RUL89xwOJKCQQZM8B
1YsYOqeL3HGxKfpFo7kmArXSRKRHToXuBgDq07KS/jxaS1a1Paz/tvYHjLxw
Y0Ot3kS+cnBeq/FGSNL/fFV3J2a8eVvydsKat3XZS3WKcNNjY2ZEY1rHgcGL
5bhVHs67bxb/IGQleyY+EwLuv5eUwS3wljJkGcWeFhlqxNXQ6NDTzRNlBS0W
4CkNiDBMegCcOlPKC2ZLGw2ejgr2utoNfmRtehr+3LAhLMVjLyPSRQ/zDhHj
Xu+Kmt4elmTmqLgAUskiOiLYpr0zI7Pb4xsEkcxRFX9rKy5WV7NhJ1lR7BKy
alO94jWIL4kJmh4GoUEhO+vDCNtW49PEgQkundV8vmzxKarUHZ0xr4feL1ZJ
THinyUs/KUAJAZSAQ1Zx/S4dNj1HuchZzDDm/nE/Y3DeDhhNUwpggmesLDxF
tqJJ/BRn8cgwM6/SMFDWUnhkX/t8qJrHphcxBjAmIdIWxDi2d78LA6xhEPUw
NdPPhUrJcu5hvhDVXcceZLa+rJEmn4aftHm6/Q06WH7dq4RaaJePP6WHvQDp
zZJOIMSEisApfh3QvHqdbiybZdyErz+yXjPXlKWG90kOz6fx+GbvGcHqibb/
HUfcDosYA7lY4xY17llY5sibvWM91ohFN5jyDlHtngi7nWQgFcDNfSh77TDT
zltUp9NnSJSgNOOwoSSNWadm6+AgbXfQNX6oJFaU4LQiAsRNa7vX/9jRfi65
5uvujM4ob199CZVxEls10UI9pIemAQQ8z/3rgQ3eyL+fViyztUPg/2IvxOHv
eexE4owH4Fo/bRlhZK0mYIamVxsRADBuBlGqx1b0OuF4AoZZgUM4d8v3iyUu
feh0QQqOkvJK/svkYHn3mf4JlUb2MTgtRQNYdZKDRgF3Q0IJaZuMyPWFsSNT
YauWjMVqnj0AEDHh6QUMF8bXLM0jGwANP+r4yPdKJNsoZMpuVoUBJYWnDTV+
8Ive6ZgBi4EEbPbMLXuqDMpDi4XcLE0UUPJ8VnmO5fAHMQkA64esY2QqldZ+
5gEhjigueZjEf0917/X53ZYWJIRiICnmYPoM0GSYJRE0k3ycdlzZzljIGk+P
Q7WgeJhthisEBDbgTuppqKNXLbNZZG/VaTdbpW1ylBv0eqamFOmyrTyh1APS
Gn37comTI3fmN6/wmVnmV4/FblvVwLuDvGgSCGPOF8i6FVfKvdESs+yr+1AE
DJXfp6h0eNEUsM3gXaJCknGhnt3awtg1fSUiwpYfDKZxwpPOYUuer8Wi+VCD
sWsUpkMxhhRqOBKaQaBDQG+kVJu6aPFlnSPQQTi1hxLwi0l0Rr38xkr+lHU7
ix8LeJVgNsQdtxbovE3i7z3ZcTFY7uJkI9j9E0muDN9x8y/YN25rm6zULYaO
jUoP/7FQZsSgxPIUvUiXkEq+FU2h0FqAC7H18cr3Za5x5dpw5nwawMArKoqG
9qlhqc34lXV0ZYwULu58EImFIS8+kITFuu7jOeSXbBgbhx8zGPqavRXeiu0t
bJd0gWs+YgMLzXtQIbQuVZENMxJSZB4aw5lPA4vr1fFBsiU4unjOEo/XAgwr
Tc0w0UndJFPvXRr3Ir5rFoIEOdRo+6os5DSlk82SBnUjwbje7BWsxWMkVhYO
6bOGUm4VxcKWXu2jU66TxQVIHy7WHktMjioVlWJdZC5Hq0g1LHg1nWSmjPY2
c/odZqN+dBBC51dCt4oi5UKmKtU5gjZsRSTcTlfhGUd6DY4Tp3CZhHjQRH4l
Zhg0bF/ooPTxIjLKK4r0+yR0lyRjqIYEY27HJMhZDXFDxBQQ1UkUIhAvXacD
WB2pb3YyeSQjt8j/WSbQY6TzdLq8SreZiuMWcXmQk4EH3xu8bPsHlcvRI+B3
gxKeLnwrVJqVLkf3m2cSGnWQhSLGbnAtgQPA6z7u3gGbBmRtP0KnAHWSK7q6
onMoYTH+b5iFjCiVRqzUBVzRRKjAL4rcL2nYeV6Ec3PlnboRzJwZIjD6i7WC
dcxERr4WVOjOBX4fhhKUiVvlmlcu8CkIiSnZENHZCpI41ypoVqVarHpqh2aP
/PS624yfxx2N3C2ci7VIuH3DcSYcaTXEKhz/PRLJXkRgVlWxn7QuaJJzDvpB
oFndoRu1+XCsup/AtkLidsSXMFTo/2Ka739+BgYDuRt1mE9EyuYyCMoxO/27
sn1QWMMd1jtcv8Ze42MaM4y/PhAMp2RfCoVZALUS2K7XrOLl3s9LDFOdSrfD
8GeMciBbfLGoXDvv5Oqq0S/OvjdID94UMcadpnSNsist/kcJJV0wtRGfALG2
+UKYzEj/2TOiN75UlRvA5XgwfqajOvmIIXybbdhxpjnSB04X3iY82TNSYTmL
LAzZlX2vmV9IKRRimZ2SpzNpvLKeB8lDhIyGzGXdiynQjFMNcVjZlmWHsH7e
ItAKWmCwNkeuAfFwir4TTGrgG1pMje7XA7kMT821cYbLSiPAwtlC0wm77F0T
a7jdMrLjMO29+1958CEzWPdzdfqKzlfBzsba0+dS6mcW/YTHaB4bDyXechZB
k/35fUg+4geMj6PBTqLNNWXBX93dFC7fNyda+Lt9cVJnlhIi/61fr0KzxOeX
NKgePKOC3Rz+fWw7Bm58FlYTgRgN63yFWSKl4sMfzihaQq0R8NMQIOjzuMl3
Ie5ozSa+y9g4z52RRc69l4n4qzf0aErV/BEe7FrzRyWh4PkDj5wy5ECaRbfO
7rbs1EHlshFvXfGlLdEfP2kKpT9U32NKZ4h+Gr9ymqZ6isb1KfNov1rw0KSq
YNP+EyWCyLRJ3EcOYdvVwVb+vIiyzxnRdugB3vNzaNljHG5ypEJQaTLphIQn
lP02xcBpMNJN69bijVtnASN/TLV5ocYvtnWPTBKu3OyOkcflMaHCEUgHPW0f
mGfld4i9Tu35zrKvTDzfxkJX7+KJ72d/V+ksNKWvwn/wvMOZsa2EEOfdCidm
oql027IS5XvSHynQtvFmw0HTk9UXt8HdVNTqcdy/jUFmXpXNP2Wvn8PrU2Dh
kkIzWhQ5Rxd/vnM2QQr9Cxa2J9GXEV3kGDiZV90+PCDSVGY4VgF8y7GedI1h
"""

In [8]:
import base64
data = base64.b64decode(ex10data)
testString = b"9AuhqAjjiKLU&>\rY5LtmCn`\rb\x0bU ga&,jg\r\x0bJv|6h1%7Q'w\x0c0gLRe\x0c^jzO.6UhPp$|fij6D\rk\x0c(rvH\x0c'pe3VPnQn[\nyJz\\3-nYJFJ(A[CxGt2~Fq\\ V\n&=#>[hTt9EM>#O!,mX*jF0D%r{PL$6yB:}PZ+]4#hN3}"
testKey = b"YELLOW SUBMARINE"
testIV = b"0000000000000000"

def encryptAES_CBC(data, key, iv):
    acc = iv
    shapedData = reshape(data, len(key))
    shapedAcc = reshape(acc, len(key))
    # print(f"Encrypting blocks = {len(shapedData)}")
    for block in shapedData:
        newBlock = xorBytes(block, shapedAcc[-1])
        # print(f"new_block ({len(newBlock)}): {newBlock}")
        newCTBlock = encryptAES_ECB(newBlock, key)
        # print(f"new_ct ({len(newCTBlock)}): {newCTBlock}")
        shapedAcc.append(newCTBlock)

    return flatten(shapedAcc)


def decryptAES_CBC(data, key):
    pt = b''
    shapedCt = reshape(data, len(key))
    shapedPt = reshape(pt, len(key))
    for blockIndex in range(len(shapedCt) - 1):
        decryptedBlock = decryptAES_ECB(shapedCt[blockIndex + 1], key)
        newPtBlock = xorBytes(decryptedBlock, shapedCt[blockIndex] )
        shapedPt.append(newPtBlock)
    return flatten(shapedPt)

#testing
ciphertext = encryptAES_CBC(testString, testKey, testIV)
plaintext = decryptAES_CBC(ciphertext,testKey)
assert plaintext == testString

#decoding
recoveredPlaintext = decryptAES_CBC(data, testKey)

print(recoveredPlaintext.decode())

 ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 

I'm like Samson

# 11. An ECB/CBC detection oracle
Now that you have ECB and CBC working:

Write a function to generate a random AES key; that's just 16 random bytes.

Write a function that encrypts data under an unknown key --- that is, a function that generates a random key and encrypts under it.

The function should look like:

encryption_oracle(your-input)
=> [MEANINGLESS JIBBER JABBER]
Under the hood, have the function append 5-10 bytes (count chosen randomly) before the plaintext and 5-10 bytes after the plaintext.

Now, have the function choose to encrypt under ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). Use rand(2) to decide which to use.

Detect the block cipher mode the function is using each time. You should end up with a piece of code that, pointed at a block box that might be encrypting ECB or CBC, tells you which one is happening.

In [9]:
#@title support fuctions
def hammingDist(bytes1, bytes2):
  result = 0
  for bit1, bit2 in zip(bytes1, bytes2):
    result += bin(bit1 ^ bit2).count('1')
  return result

b1 = b"this is a test"
b2 = b"wokka wokka!!!"
assert hammingDist(b1,b2) == 37

def normHammingDistance(bytes1, bytes2):
  length1 = len(bytes1)
  length2  = len(bytes2)
  assert length1 == length2, f"bytes1 len {length1}, bytes2 len {length2}"
  return hammingDist(bytes1, bytes2) / length1

def guessKeySize(data):
    maxKeySize = min(len(data) // 4, 50)

    keyScores = []

    for keySize in range(1, maxKeySize):
        maxFrames = len(data) // keySize

        frames = [data[i*keySize : (i+1)*keySize] for i in range(maxFrames)]
        pairs = list(zip(frames[:-1], frames[1:]))

        normDists = list(map(lambda t : normHammingDistance(t[0],t[1]), pairs))
        score = sum(normDists) / len(normDists)

        keyScores.append((score, keySize))
    #we want a low Hamming distance
    keyScores = sorted(keyScores)
    return keyScores

In [10]:
import random

def randomBytes(length = 10):
    values = [random.randrange(0,256) for i in range(length)]
    return bytes(values)

def encryptWithUnknownKey(data):
    key = randomBytes(16)
    header = randomBytes(random.randint(5,10))
    footer = randomBytes(random.randint(5,10))

    enclosedData = header + data + footer

    if random.randint(0,1) == 0:
        #encrypt ECB
        print("ECB")
        return encryptAES_ECB(enclosedData, key)
    else:
        #encrypt CBC
        print("CBC")
        return encryptAES_CBC(enclosedData, key, randomBytes(16))

def keysizeFor_AES_ECB(ciphertext):
    score, keySize = guessKeySize(ciphertext)[0]
    if score < 3:
        return True, keySize
    else:
        return False, 0

In [11]:
def detectAES_ECB(encryptionFunction, n = 10):
    for i in range(n):
        #encrypt some 0 bytes and measure the Hamming distance between adjacent
        #blocks. A normalised Hamming distance lower than 2 indicates ECB
        isECB, guessedKeySize = keysizeFor_AES_ECB(encryptionFunction(bytes(100)))
        if isECB:
            print(f"encryption was probably ECB; key length: {guessedKeySize} bytes ({guessedKeySize * 8} bits)\n")
        else:
            print(f"encryption was probably CBC\n")

detectAES_ECB(encryptWithUnknownKey, 10)

CBC
encryption was probably CBC

ECB
encryption was probably ECB; key length: 16 bytes (128 bits)

CBC
encryption was probably CBC

ECB
encryption was probably ECB; key length: 16 bytes (128 bits)

CBC
encryption was probably CBC

ECB
encryption was probably ECB; key length: 16 bytes (128 bits)

CBC
encryption was probably CBC

ECB
encryption was probably ECB; key length: 16 bytes (128 bits)

ECB
encryption was probably ECB; key length: 16 bytes (128 bits)

ECB
encryption was probably ECB; key length: 16 bytes (128 bits)



# 12. Byte-at-a-time ECB decryption (Simple)
Copy your oracle function to a new function that encrypts buffers under ECB mode using a consistent but unknown key (for instance, assign a single random key, once, to a global variable).

Now take that same function and have it append to the plaintext, BEFORE ENCRYPTING, the following string:

``Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK``

## Spoiler alert.
Do not decode this string now. Don't do it.

Base64 decode the string before appending it. Do not base64 decode the string by hand; make your code do it. The point is that you don't know its contents.

What you have now is a function that produces:

``AES-128-ECB(your-string || unknown-string, random-key)``
It turns out: you can decrypt "unknown-string" with repeated calls to the oracle function!

## Here's roughly how:

Feed identical bytes of your-string to the function 1 at a time --- start with 1 byte ("A"), then "AA", then "AAA" and so on. Discover the block size of the cipher. You know it, but do this step anyway.
Detect that the function is using ECB. You already know, but do this step anyways.
Knowing the block size, craft an input block that is exactly 1 byte short (for instance, if the block size is 8 bytes, make "AAAAAAA"). Think about what the oracle function is going to put in that last byte position.
Make a dictionary of every possible last byte by feeding different strings to the oracle; for instance, "AAAAAAAA", "AAAAAAAB", "AAAAAAAC", remembering the first block of each invocation.
Match the output of the one-byte-short input to one of the entries in your dictionary. You've now discovered the first byte of unknown-string.
Repeat for the next byte.

## Congratulations.
This is the first challenge we've given you whose solution will break real crypto. Lots of people know that when you encrypt something in ECB mode, you can see penguins through it. Not so many of them can decrypt the contents of those ciphertexts, and now you can. If our experience is any guideline, this attack will get you code execution in security tests about once a year.

In [12]:
#@title data
ex12Key = b'\x19\x1b\xac\x8aR\x0bNt\x079\xb9\xb8\x80X\xd6\xc0'
suffixData = """Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK
"""

In [13]:
#encrypt with a fixed key after appending a known suffix
def AES_128_ECB_Suffix(data):
    suffix = base64.b64decode(suffixData)
    enclosedData = data + suffix
    return encryptAES_ECB(enclosedData, ex12Key)

#detect ECB block size
def blockSizeDetector_AES(encryptionFunction):
    lastCiphertextLength = 0
    candidateBlockSize = 0

    for length in range(32):
        plaintext = bytes(length)
        nextCiphertextLength = len(AES_128_ECB_Suffix(plaintext))
    
        if nextCiphertextLength > lastCiphertextLength:
            candidateBlockSize = nextCiphertextLength - lastCiphertextLength
            lastCiphertextLength = nextCiphertextLength
    return candidateBlockSize

blockSizeBytes = blockSizeDetector_AES(AES_128_ECB_Suffix)
print(f"block size: {blockSizeBytes} bytes ({blockSizeBytes * 8} bits)")

#detect ECB mode (this also prints the key length: 16)
detectAES_ECB(AES_128_ECB_Suffix, 1)

block size: 16 bytes (128 bits)
encryption was probably ECB; key length: 16 bytes (128 bits)



In [48]:
import string

def getSuffixByte(encryptionFuction, guessedBytes, byteNumber):
    blockSize = 16 #bytes
    blockOfDesiredByte = byteNumber // blockSize
    blockStartIndex = blockSize * blockOfDesiredByte
    blockEndIndex = blockStartIndex + blockSize

    #maps candidate bytes to ciphertext blocks
    dictionaryAcc = {}
    
    padding = bytes((blockSize - byteNumber % blockSize) - 1)
    CtBlock = encryptionFuction(padding)[blockStartIndex : blockEndIndex]

    #build a plaintext 1 byte shorter than the cipher block size
    PtShort = padding + guessedBytes
    assert len(PtShort) % blockSize == blockSize - 1, f"wrong pt size {len(PtShort)}, guessedBytes: {guessedBytes}"

    # append all possible bytes to the end, encrypt them and store the results
    for character in range(256):
        plaintext = PtShort + bytes([character])
        assert len(plaintext) % blockSize == 0
        ct = encryptionFuction(plaintext)
        dictionaryAcc[character] = ct[blockStartIndex : blockEndIndex]

    #find a block in the dictionary that matches the encrypted padding.
    for character, block in dictionaryAcc.items():
        if block == CtBlock:
            return bytes([character])
    return bytes('?', 'ascii')


decodedSuffix = b''
for i in range(len(suffixData)):
    decodedSuffix += getSuffixByte(AES_128_ECB_Suffix, decodedSuffix, i)

print(f"length: {len(decodedSuffix)}\n{decodedSuffix.decode()}")

length: 188
Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by
???????????????????????????????????


# 13. ECB cut-and-paste
Write a k=v parsing routine, as if for a structured cookie. The routine should take:

``
foo=bar&baz=qux&zap=zazzle
``

... and produce:

``
{
  foo: 'bar',
  baz: 'qux',
  zap: 'zazzle'
}
``
(you know, the object; I don't care if you convert it to JSON).

Now write a function that encodes a user profile in that format, given an email address. You should have something like:

``
profile_for("foo@bar.com")
``

... and it should produce:

``
{
  email: 'foo@bar.com',
  uid: 10,
  role: 'user'
}
``

... encoded as:

``
email=foo@bar.com&uid=10&role=user
``

Your "profile_for" function should not allow encoding metacharacters (& and =). Eat them, quote them, whatever you want to do, but don't let people set their email address to ``"foo@bar.com&role=admin"``.

Now, two more easy functions. Generate a random AES key, then:

* Encrypt the encoded user profile under the key; "provide" that to the "attacker".
* Decrypt the encoded user profile and parse it.

Using only the user input to profile_for() (as an oracle to generate "valid" ciphertexts) and the ciphertexts themselves, make a role=admin profile.